In [114]:
from google.colab import userdata
key=userdata.get('COHERE_API_KEY')

In [129]:
!pip install lancedb sentence-transformers polars tantivy cohere langchain_cohere -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.6/377.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00


In [96]:
import os
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
import polars as pl
import pandas as pd

db = lancedb.connect("/tmp/db")
model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5", device="cpu")

In [104]:
if os.path.exists("/tmp/db"):
    !rm -rf /tmp/db

In [105]:
class Documents(LanceModel):
    text: str = model.SourceField()
    vector: Vector(model.ndims()) = model.VectorField()

table = db.create_table("pl_table", schema=Documents)

In [145]:
df = pd.read_csv("data.csv")
print(df)

     name                                        description industry  \
0  GitLab  GitLab is the first single application for the...      B2B   

                          category                 url           headline  \
0  Engineering, Product and Design  http://gitlab.com/  Software. Faster.   

                                             subhead             cta  founded  \
0  GitLab is the most comprehensive AI-powered De...  Get free trial     2012   

        location                                               tags  
0  San Francisco  W15, PUBLIC, DEVELOPER-TOOLS, DEVSECOPS, OPEN-...  


In [146]:
df["text"] = df[df.columns.to_list()].astype(str).agg(" ".join, axis=1)

docs = [{"text": x} for x in df["text"]]
docs

[{'text': 'GitLab GitLab is the first single application for the entire DevOps lifecycle. Only GitLab enables Concurrent DevOps, unlocking organizations from the constraints of today’s toolchain. GitLab provides unmatched visibility, radical new levels of efficiency and comprehensive governance to significantly compress the time between planning a change and monitoring its effect. This makes the software lifecycle 200% faster, radically improving the speed of business. GitLab and Concurrent DevOps collapses cycle times by driving higher efficiency across all stages of the software development lifecycle. For the first time, Product, Development, QA, Security, and Operations teams can work concurrently in a single application. There’s no need to integrate and synchronize tools, or waste time waiting for handoffs. Everyone contributes to a single conversation, instead of managing multiple threads across disparate tools. And only GitLab gives teams complete visibility across the lifecycle 

In [108]:
table.add(docs)

In [109]:
table.create_fts_index("text")

In [115]:
from lancedb.rerankers import CohereReranker

reranker = CohereReranker(api_key=key)

In [147]:
query = "How many repositories does github have?"

In [136]:
result = table.search(query, query_type="hybrid").limit(5).rerank(reranker=reranker).to_list()

In [137]:
context = result[0]["text"]
context

'GitLab GitLab is the first single application for the entire DevOps lifecycle. Only GitLab enables Concurrent DevOps, unlocking organizations from the constraints of today’s toolchain. GitLab provides unmatched visibility, radical new levels of efficiency and comprehensive governance to significantly compress the time between planning a change and monitoring its effect. This makes the software lifecycle 200% faster, radically improving the speed of business. GitLab and Concurrent DevOps collapses cycle times by driving higher efficiency across all stages of the software development lifecycle. For the first time, Product, Development, QA, Security, and Operations teams can work concurrently in a single application. There’s no need to integrate and synchronize tools, or waste time waiting for handoffs. Everyone contributes to a single conversation, instead of managing multiple threads across disparate tools. And only GitLab gives teams complete visibility across the lifecycle with a sin

In [148]:
from langchain_cohere import ChatCohere

prompt = f"""
Context is given below:
{context}

Answer the question using the context: {query}
"""

cohere_chat_model = ChatCohere(cohere_api_key=key)
response = cohere_chat_model.invoke(prompt)
print(f"Response: \n {response.content}")

Response: 
 I apologize, but I cannot answer your question based on the provided context. The context provided is about GitLab, which is a single application for the entire DevOps lifecycle. It does not mention anything about GitHub or the number of repositories it has.


LOL!